In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils_nn as utlnn


## Uso de funciones Personalizadas cont tf.map_fn
Al tener funciones personalizadas, podemos usar tf.map_fn para aplicar a cada elemento del tensor.

In [2]:
x_test = pd.read_csv("./Train_Test/x_test.csv")
y_test = pd.read_csv("./Train_Test/y_test.csv")

autoencoder_Lat_Lon = tf.keras.models.load_model('modelos_entrenamiento/modelos_latitudes_longitudes/entrenamiento_lat_lon_600_120.keras')
autoencoder_Height = tf.keras.models.load_model('modelos_entrenamiento/entrenamiento_alturas2.keras')

In [ ]:
lat_pred, lon_pred = autoencoder_Lat_Lon.predict(x_test)
height_pred = autoencoder_Height.predict(x_test)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [4]:
y_true_lat = y_test.iloc[:,:100]
y_true_lon = y_test.iloc[:,100:200]
y_true_height = y_test.iloc[:,200:300] #km
print("Formatos True y Pred: ",type(y_true_lat), type(lat_pred))
y_true_lat = y_true_lat.to_numpy()
y_true_lon = y_true_lon.to_numpy()
y_true_height = y_true_height.to_numpy()
print("Formatos True y Pred: ",type(y_true_lat), type(lat_pred))


Formatos True y Pred:  <class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
Formatos True y Pred:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
distancias_2d = np.array(list(map(lambda args: np.array(utlnn.distances_by_geodesic(*args)), zip(y_true_lat, y_true_lon, lat_pred, lon_pred))))

In [27]:
print(distancias_2d[0])

[ 9.13053263 13.56368761  2.32750364  9.55844824  4.36794193 17.22881261
  9.00712572 10.38056692 20.04029248 15.90541073  2.7102068  12.7110335
 21.98011402  6.89573507  7.6170004  31.22027428 15.91487706 10.65131627
 10.11685305  6.02565304 26.03155305  6.37794531 20.3526025  19.26476537
 22.4612194  18.94365867 16.26637931 16.46475888 26.23756044 18.51066522
 17.82496059 20.87323629 25.30077567 19.99658959 30.38121702 23.67908504
 16.77407786 12.58654666 14.86357188 19.85009024 26.66402713 15.75428396
 22.25711343 22.1365469   8.70863119 24.88740278 21.6599953  13.13754548
 30.71169323 28.96174444 23.38294918 26.11160668 29.63989092 24.8874294
 27.72904312 22.29664664 36.4206842  26.35361639 41.70396141 24.88308324
 24.03763928 32.64597355 29.78574268 41.28902362 19.74467218 21.39430594
 38.77918103 29.09534864 25.40856497 19.92754948 25.1378535  29.77335102
 28.24496286 37.96577421 29.34331788 35.2019233  30.94118277 27.83495904
 39.53526945 41.12097573 27.24055566 32.10429903 27.0